# DreamBooth - Easy Run (Xavier Geerinck - https://xaviergeerinck.com)

This notebook provides an updated implementation from https://github.com/TheLastBen/fast-stable-diffusion, focusing on easiness of running and being able to run it locally. If you face any issues, feel free to publish a comment on the blog https://xaviergeerinck.com

Some of the optimizations added:
- Removed the need to download dependencies as zip and extract them, rather focus on pip install to work cross architecture
- Removed the need for Python3.8 only (later versions have more optimizations)

## Download and install diffusers

In [1]:
!git clone https://github.com/huggingface/diffusers; cd diffusers; pip install -e .

Cloning into 'diffusers'...
remote: Enumerating objects: 15199, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 15199 (delta 110), reused 116 (delta 78), pack-reused 15036
Receiving objects: 100% (15199/15199), 10.28 MiB | 9.34 MiB/s, done.
Resolving deltas: 100% (10482/10482), done.
/Users/xaviergeerinck/Projects/PublicProjects/AI/stable-diffusion/diffusers
Obtaining file:///Users/xaviergeerinck/Projects/PublicProjects/AI/stable-diffusion/diffusers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for diffusers (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.10.2-0.editable-py3-none-any.whl size=14810 sha256=2ffa9a63285b876f6b48ea37dc0ee49b162eb6557d561ab5738021228d5c40ad
  Stored in directory: /private/var/folde

## Install Dreambooth Example

In [3]:
!cd diffusers/examples/dreambooth && pip install -r requirements.txt

/Users/xaviergeerinck/Projects/PublicProjects/AI/stable-diffusion
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 7.1 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 7.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 9.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 KB 8.3 MB/s eta 0:00:00
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached wheel-0.38.4-py3-none-any.whl (36 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.0/177.0 KB 7.0 MB/s eta 0:00:00
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 KB 7.1 MB/s eta 0:00:00
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached re

## Initialize Accelerate Environment

In [8]:
from accelerate.utils import write_basic_config
write_basic_config()

/Users/xaviergeerinck/.pyenv/versions/3.10.5/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Configure Dataset

In [ ]:
# Download the following images to ./dataset
# https://drive.google.com/drive/folders/1BO_dyz-p65qhBRRMRA4TbZ8qW4rB99JZ

## Run Training

TODO:
* Optimize this for avatar faces
* Check https://huggingface.co/blog/dreambooth#fine-tuning-the-text-encoder to see what happens if we fine tune with DDIM
  * https://github.com/huggingface/diffusers/tree/main/examples/dreambooth#fine-tune-text-encoder-with-the-unet
* Make sure we provide 768x768 images as input
* See if huggingface-cli login is required each time?
* See what `pip install --upgrade git+https://github.com/huggingface/diffusers.git transformers accelerate scipy`

In [13]:
%%bash
# Note: use resolution 768 when using stable diffusion 2
# stabilityai/stable-diffusion-2
# !export MODEL_NAME="CompVis/stable-diffusion-v1-4"
export MODEL_NAME="stabilityai/stable-diffusion-2"
export INSTANCE_DIR="../../../dataset"
export OUTPUT_DIR="../../../model"

cd diffusers/examples/dreambooth

accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --instance_prompt="a photo of sks dog" \
  --resolution=768 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=400

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_cpu_threads_per_process` was set to `10` to improve out-of-box performance
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Downloading: 100%|██████████| 824/824 [00:00<00:00, 277kB/s]
Downloading: 100%|██████████| 1.06M/1.06M [00:01<00:00, 976kB/s] 
Downloading: 100%|██████████| 525k/525k [00:00<00:00, 749kB/s] 
Downloading: 100%|██████████| 633/633 [00:00<00:00, 660kB/s]
You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.


Error while terminating subprocess (pid=90485): 
